In [10]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langchain_experimental langsmith pandas langgraph

In [11]:
%pip install --upgrade --quiet  google-search-results

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("SERP_API_KEY")
_set_if_undefined("SERPAPI_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Supervisor"

Please provide your OPENAI_API_KEY ········
Please provide your LANGCHAIN_API_KEY ········
Please provide your TAVILY_API_KEY ········
Please provide your SERP_API_KEY ········
Please provide your SERPAPI_API_KEY ········


In [2]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain.agents import load_tools

tavily_tool = TavilySearchResults(max_results=5)

google_finance_tools = load_tools(["google-scholar", "google-finance"])

dalle_image_generator_tools = load_tools(["dalle-image-generator"])

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [4]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [5]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "ImageGenerator" , "FinanceAdvisor"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [6]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

image_generator_agent = create_agent(
    llm,
    dalle_image_generator_tools,
    "You may generate images based on the description.",
)
image_generator_node = functools.partial(agent_node, agent=image_generator_agent, name="ImageGenerator")

finance_advisor_agent = create_agent(
    llm,
    google_finance_tools,
    "You are a financial advisor.",
)
finance_advisor_node = functools.partial(agent_node, agent=finance_advisor_agent, name="FinanceAdvisor")


workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("ImageGenerator", image_generator_node)
workflow.add_node("FinanceAdvisor", finance_advisor_node)
workflow.add_node("supervisor", supervisor_chain)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [8]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on Bogazici University.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content="Bogazici University is a prominent public research university located in Istanbul, Turkey. It was founded in 1971, inheriting its main campus from a former American educational institution, Robert College, which is reflected in the fact that classes at Bogazici University are taught in English. The university's historical roots can be traced back to its origin as Robert College, the first American higher education institution founded outside the United States.\n\nAs of the latest information, Bogazici University has made significant progress in global university rankings, such as rising almost 300 places in the QS Sustainability 2024 Ranking and improving its position in the THE World University Rankings 2024. The university is considered one of the top 500 universities in the world according to the 2018 edition of the QS World University Rankings.\n\nThe university's academic structure inclu

In [9]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Generate an image for a happy human")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'ImageGenerator'}}
----
{'ImageGenerator': {'messages': [HumanMessage(content='Here is an image of a happy human:\n\n![Happy Human](https://oaidalleapiprodscus.blob.core.windows.net/private/org-pbaCkW49bTYFJQz3RIDIduq8/user-4BtnWKFpRLqi7szZyLy7DAZk/img-uZkqsR2oUrklio4drseSmZj0.png?st=2024-03-25T17%3A40%3A06Z&se=2024-03-25T19%3A40%3A06Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-03-25T14%3A52%3A14Z&ske=2024-03-26T14%3A52%3A14Z&sks=b&skv=2021-08-06&sig=BjaVt1M3tzCTausJCAIUMCz6qyW90/0BtbuIMWJ55SU%3D)', name='ImageGenerator')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [10]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Should I buy google stocks based on the last 30 days prices")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'FinanceAdvisor'}}
----
{'FinanceAdvisor': {'messages': [HumanMessage(content="As a financial advisor, I must stress that investment decisions should not be based solely on the stock price movements of the last 30 days. While it's important to consider recent price trends, you should also evaluate a range of factors before deciding whether to buy Google (Alphabet Inc.) stocks or any other investment. Here are some key considerations:\n\n1. **Company Fundamentals**: Analyze Google's financial health by reviewing its balance sheet, income statement, and cash flow statement. Look for trends in revenue, net income, profitability margins, and return on equity.\n\n2. **Growth Prospects**: Consider Google's potential for future growth. This includes looking at its product pipeline, market share, competitive advantages, and expansion plans.\n\n3. **Market Conditions**: Assess the overall market conditions and economic indicators. Are we in a bull market, bear market, or

In [12]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Find how much google stock's value has increased/decreased today. Then generate an image of a human. If the stock price has increased , it is a happy human. If the stock price has decreased , it is a sad human.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'FinanceAdvisor'}}
----
{'FinanceAdvisor': {'messages': [HumanMessage(content="The stock value of Google (GOOGL) on NASDAQ has decreased today by 0.81%. Based on this information, I will generate an image of a sad human to reflect the decrease in stock value.\n\n[Image generation is not possible in this text-based environment, but normally, at this point, an image of a sad human would be created to visually represent the decrease in Google's stock value.]", name='FinanceAdvisor')]}}
----
{'supervisor': {'next': 'ImageGenerator'}}
----
{'ImageGenerator': {'messages': [HumanMessage(content="Based on the decrease in Google's stock value today, here is an image of a sad human:\n\n![Sad Human](https://oaidalleapiprodscus.blob.core.windows.net/private/org-pbaCkW49bTYFJQz3RIDIduq8/user-4BtnWKFpRLqi7szZyLy7DAZk/img-Rgwyt0SgfaH8oclQLtHD9U2e.png?st=2024-03-25T17%3A44%3A27Z&se=2024-03-25T19%3A44%3A27Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4